# Analisis de superficial del csv de auctions
### En este notebook exploramos los registros del csv con el objetivo de buscar relaciones entre las distintas variables

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Leemos el csv, y estalbecemos el tipo de dato de cada columna

In [ ]:
auctions = pd.read_csv("../data/auctions.csv", dtype={'date': 'str','device_id': 'category', 'ref_type_id': 'category','source_id':'category'}, parse_dates = ['date'])
auctions['day'] = auctions['date'].dt.date
auctions['hour'] = auctions['date'].dt.hour
auctions.rename(columns = {"device_id": "ref_hash"}, inplace = True)
auctions.head(5)

## Elegimos las columnas con menor entropia o pocos valores diferentes y nos quedamos con los 5 valores mas comunes

In [ ]:
#grupos = ['ref_type_id', 'source_id', 'hour', 'day']
#top5 = {}
#for i in grupos: 
#    top5[i]= auctions[i].value_counts().head(5).index.tolist()
#top5
auctions=auctions.drop(['ref_type_id','source_id'],axis=1)

## Solo correr uno de los siguientes a la vez, es muy grande, y el codigo esta diseniado para solo realizar uno

In [ ]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')
auctions=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )]
numero_de_ventana = 7

In [ ]:
#auctions = auctions.sort_values(by = 'date')
#auctions = auctions.sort_values(by = 'ref_hash')
auctions['repeticiones'] = 1
device_ids = auctions['ref_hash'].to_frame().set_index('ref_hash')
#auctions = pd.get_dummies(auctions, columns = grupos)
for column in grupos:
    for value in top5[column]:
        auctions[column + str(value)] = (auctions[column] == value)*1

## Esto si se corre siempre

In [ ]:
features = ['sum', 'mean', 'std', 'min', 'max']

features_ventana = auctions.groupby('ref_hash').agg(features).fillna(0)
level0 = features_ventana.columns.get_level_values(0)
level1 = features_ventana.columns.get_level_values(1)
features_ventana.columns = level0 + "_" + level1
features_ventana = features_ventana.loc[features_ventana['repeticiones_sum'] > 0]
features_ventana.to_csv('../xgb/features_auctions_gonzalo_ventana' + str(numero_de_ventana) + '.csv')